**PRIMEIRA ETAPA**

In [1]:
#Instalando pyspark no colab
!pip install pyspark

In [2]:
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, sum

In [3]:
#Criando sessão spark
spark = SparkSession.builder.getOrCreate()

In [4]:
#Montando Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Carregando dataset
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/projetoChurn/Telco-Churn.csv", header=True, inferSchema=True)

In [21]:
#visualizando Schema
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: float (nullable = false)
 |-- Churn: string (nullable = true)



In [7]:
#Removendo espaços em branco e convertendo a coluna TotalCharges para númerico
df = df.withColumn("TotalCharges", trim(col("TotalCharges")).cast("float"))

In [8]:
#Conta nulos por coluna
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [9]:
#Mudando valores nulos na coluna TotalCharges
df = df.na.fill({"TotalCharges": 0})

In [10]:
#filtrando os valores modificados
df.filter(df.TotalCharges == 0).select("tenure", "MonthlyCharges", "TotalCharges").show()

+------+--------------+------------+
|tenure|MonthlyCharges|TotalCharges|
+------+--------------+------------+
|     0|         52.55|         0.0|
|     0|         20.25|         0.0|
|     0|         80.85|         0.0|
|     0|         25.75|         0.0|
|     0|         56.05|         0.0|
|     0|         19.85|         0.0|
|     0|         25.35|         0.0|
|     0|          20.0|         0.0|
|     0|          19.7|         0.0|
|     0|         73.35|         0.0|
|     0|          61.9|         0.0|
+------+--------------+------------+



In [11]:
#Estatísticas descritivas
df.describe().show()

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7043|     

In [12]:
#frequência de churn
df.groupBy("Churn").count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5174|
|  Yes| 1869|
+-----+-----+



In [13]:
#Distribuição de contratos
df.groupBy("Contract").count().show()

+--------------+-----+
|      Contract|count|
+--------------+-----+
|Month-to-month| 3875|
|      One year| 1473|
|      Two year| 1695|
+--------------+-----+



In [14]:
#fazendo correlaçoes em pyspark
print("Corr(SeniorCitizen, tenure):", df.corr('SeniorCitizen', 'tenure'))
print("Corr(SeniorCitizen, MonthlyCharges):", df.corr('SeniorCitizen', 'MonthlyCharges'))
print("Corr(SeniorCitizen, TotalCharges):", df.corr('SeniorCitizen', 'TotalCharges'))
print("Corr(tenure, MonthlyCharges):", df.corr('tenure', 'MonthlyCharges'))
print("Corr(tenure, TotalCharges):", df.corr('tenure', 'TotalCharges'))
print("Corr(MonthlyCharges, TotalCharges):", df.corr('MonthlyCharges', 'TotalCharges'))

Corr(SeniorCitizen, tenure): 0.016566877681809318
Corr(SeniorCitizen, MonthlyCharges): 0.22017333857627205
Corr(SeniorCitizen, TotalCharges): 0.10300618432182765
Corr(tenure, MonthlyCharges): 0.24789985628615094
Corr(tenure, TotalCharges): 0.8261783979021456
Corr(MonthlyCharges, TotalCharges): 0.65117383153822


**Interpretação dos seus dados:**
Par de colunas	Correlação	Interpretação

tenure × TotalCharges	0.82	Forte relação: quanto mais tempo o cliente está, maior o total pago. Óbvio e esperado.

MonthlyCharges × TotalCharges	0.65	Boa relação: clientes que pagam mais por mês tendem a acumular mais no total.

tenure × MonthlyCharges	0.24	Fraca relação: o tempo não define o valor mensal.

SeniorCitizen × outros	0.01 a 0.22	Muito fraca: idade (ou faixa etária) tem pouco impacto direto nos valores.

In [20]:
#Modificando colunas para Int
df_clean = df.select(
    col("MonthlyCharges").cast("float"),
    col("TotalCharges").cast("float")
)

In [17]:
#Filtrando fatura mensal > R$ 80 + Menos de 12 meses de contrato
filtro = df.filter(
    (col('Churn') == 'Yes') &
    (col('MonthlyCharges') > 80) &
    (col('tenure') < 12)
)

filtro.count()

321

In [18]:
#Filtrando churn por fibra e sem suporte tecnico
df.filter((col('Churn') == 'Yes') & (col('InternetService') == 'Fiber optic')).count()
df.filter((col('Churn') == 'Yes') & (col('TechSupport') == 'No')).count()

1446

In [22]:
#Passando para pandas
import pandas as pd


In [23]:
#Exportando como csv no drive
df_clean = df.toPandas()
df_clean.to_csv('/content/drive/MyDrive/Colab Notebooks/projetoChurn/final_churn.csv', index=False)

In [24]:
spark.stop()